In [373]:
import pandas as pd

In [374]:
import numpy as np

In [375]:
from sklearn.ensemble import RandomForestRegressor

In [376]:
%pip install fredapi
from fredapi import Fred

#### **Load CSV files such as, countyData.csv, nysData.csv, propertyTaxPerCounty10years.csv**

In [377]:
county_data = pd.read_csv('countyData.csv')

In [378]:
nys_data = pd.read_csv('nysData.csv')

In [379]:
tax = pd.read_csv('propertyTaxPerCounty10years.csv')

#### **Load NYSsingleBedCountyDF.csv and NYSsingleFamilyMetroDF.csv**

In [380]:
county_prices = pd.read_csv('NYSsingleBedCountyDF.csv')

In [381]:
metro = pd.read_csv('NYSsingleFamilyMetroDF.csv')

#### **Mortgge rate**

In [382]:
fred = Fred(api_key='bd2983759a4cf1e15823ed3442667fb4') # it's the key from https://fredaccount.stlouisfed.org/apikey
rates = fred.get_series('MORTGAGE30US', observation_start='2015-01-01') #mortgage rate from 2015 to 2025
df_rates = pd.DataFrame(rates, columns=['Mortgage Rate']) # DataFrame for the fred data

#### **Clean Data Metro**

In [383]:
metro_long = metro.melt(id_vars=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'],
                        var_name='Date',
                        value_name='HomePrice')

metro_long["Date"] = pd.to_datetime(metro_long["Date"])
metro_long = metro_long.dropna()
metro_long.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,Date,HomePrice
0,394913,1,"New York, NY",msa,NY,2015-01-31,1986.778076
1,394425,50,"Buffalo, NY",msa,NY,2015-01-31,1022.171678
2,395031,52,"Rochester, NY",msa,NY,2015-01-31,1058.697013
3,394308,64,"Albany, NY",msa,NY,2015-01-31,1271.508966
4,845159,86,"Poughkeepsie, NY",msa,NY,2015-01-31,1606.776133


### **Clean COUNTY HOME PRICE DATA**

In [384]:
county_long = county_prices.melt(
    id_vars=["RegionID", "SizeRank", "RegionName", "RegionType", "StateName",
            "State", "Metro", "StateCodeFIPS", "MunicipalCodeFIPS"],
    var_name="PriceDate",
    value_name="HomePrice"
)

county_long["PriceDate"] = pd.to_datetime(county_long["PriceDate"])
county_long = county_long.dropna()


county_long["County"] = county_long["RegionName"].str.replace(" County", "")
county_long["Year"] = county_long["PriceDate"].dt.year
county_long["Month"] = county_long["PriceDate"] + pd.offsets.MonthEnd(0)


### **CLEAN 3 — PROPERTY TAX DATA**

In [385]:
tax_long = tax.melt(id_vars=['County'], var_name='Year', value_name='TaxAmount')
tax_long['TaxDate'] = pd.to_datetime(tax_long['Year'], format='%Y')
tax_clean = tax_long.drop(columns=["Year"])
tax_clean["Year"] = pd.to_datetime(tax_clean["TaxDate"]).dt.year
tax_clean.head()

,County,TaxAmount,TaxDate,Year
0,Albany,3526.0,2015-01-01,2015
1,Allegany,1959.0,2015-01-01,2015
2,Bronx,2653.0,2015-01-01,2015
3,Broome,2417.0,2015-01-01,2015
4,Cattaraugus,2054.0,2015-01-01,2015


#### **CLEAN 4 — COUNTY COST OF LIVING (RENT PROXY)**

In [386]:
county_data["RentDate"] = pd.to_datetime("2024-01-01")
county_rent = county_data[["RentDate", "County", "Housing"]].copy()

# Convert annual housing cost to monthly rent
county_rent["MonthlyRent"] = county_rent["Housing"] / 12

county_rent.head()

,RentDate,County,Housing,MonthlyRent
0,2024-01-01,Albany,13098.0,1091.500000
1,2024-01-01,Allegany,8435.0,702.916667
2,2024-01-01,Bronx,21130.0,1760.833333
3,2024-01-01,Broome,10472.0,872.666667
4,2024-01-01,Cattaraugus,8833.0,736.083333


#### **CLEAN 5 MERGE ALL COUNTY-LEVEL DATA INTO ONE MASTER DATASET**

In [387]:
#Prepare mortgage rate dataframe
df_rates = df_rates.reset_index()
df_rates = df_rates.rename(columns={"index": "RateDate"})
df_rates["RateDate"] = pd.to_datetime(df_rates["RateDate"])

# Convert weekly mortgage data to monthly average
df_rates = df_rates.resample("M", on="RateDate").mean().reset_index()

# Align county home price dates to month-end
county_long["Month"] = county_long["PriceDate"] + pd.offsets.MonthEnd(0)

# Make RateDate == Month for merging
df_rates = df_rates.rename(columns={"RateDate": "Month"})
df_rates["Month"] = df_rates["Month"] + pd.offsets.MonthEnd(0)

# Merge county home prices + taxes
county_merged = pd.merge(
    county_long,
    tax_clean,
    on=["County", "Year"],
    how="left"
)

# DELETE old mortgage rate columns if they exist
for col in ["Mortgage Rate", "Mortgage Rate_x", "Mortgage Rate_y"]:
    if col in county_merged.columns:
        county_merged = county_merged.drop(columns=[col])

# Correct Single Merge for Mortgage Rate
county_merged = county_merged.merge(
    df_rates[["Month", "Mortgage Rate"]],
    on="Month",
    how="left"
)

# Add rent data
county_merged = pd.merge(
    county_merged,
    county_rent,
    on="County",
    how="left"
)


/tmp/ipython-input-4134376902.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_rates = df_rates.resample("M", on="RateDate").mean().reset_index()


In [388]:
#county_merged

In [389]:
county_merged[county_merged["Year"] >= 2015].head(20)


,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,PriceDate,HomePrice,County,Year,Month,TaxAmount,TaxDate,Mortgage Rate,RentDate,Housing,MonthlyRent
9172,581,6,Kings County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,47,2015-01-31,411315.513185,Kings,2015,2015-01-31,2903.0,2015-01-01,3.67,2024-01-01,26251.0,2187.583333
9173,1347,10,Queens County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,81,2015-01-31,275119.382556,Queens,2015,2015-01-31,2914.0,2015-01-01,3.67,2024-01-01,28178.0,2348.166667
9174,2452,20,New York County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,61,2015-01-31,833111.529246,New York,2015,2015-01-31,5873.0,2015-01-01,3.67,2024-01-01,31371.0,2614.250000
9175,2046,24,Suffolk County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,103,2015-01-31,188032.745783,Suffolk,2015,2015-01-31,7192.0,2015-01-01,3.67,2024-01-01,22245.0,1853.750000
9176,401,26,Bronx County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,5,2015-01-31,152386.498527,Bronx,2015,2015-01-31,2653.0,2015-01-01,3.67,2024-01-01,21130.0,1760.833333
9177,1252,29,Nassau County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,59,2015-01-31,256284.902655,Nassau,2015,2015-01-31,8711.0,2015-01-01,3.67,2024-01-01,22295.0,1857.916667
9178,3148,48,Westchester County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,119,2015-01-31,220162.386541,Westchester,2015,2015-01-31,9003.0,2015-01-01,3.67,2024-01-01,27604.0,2300.333333
9179,157,55,Erie County,county,NY,NY,"Buffalo-Cheektowaga, NY",36,29,2015-01-31,83662.742520,Erie,2015,2015-01-31,3120.0,2015-01-01,3.67,2024-01-01,12016.0,1001.333333
9180,1223,87,Monroe County,county,NY,NY,"Rochester, NY",36,55,2015-01-31,73309.624571,Monroe,2015,2015-01-31,3805.0,2015-01-01,3.67,2024-01-01,12754.0,1062.833333
9181,2511,143,Richmond County,county,NY,NY,"New York-Newark-Jersey City, NY-NJ-PA",36,85,2015-01-31,236189.984726,Richmond,2015,2015-01-31,2842.0,2015-01-01,3.67,2024-01-01,24853.0,2071.083333


In [390]:
county_merged.to_csv("County_merges_clean.csv", index=False)